<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain
%pip install kor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.2/566.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

In [5]:
import json

def json_dump(json_str):
  json_object = json.loads(json_str)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)

In [ ]:
import os

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-3",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)


In [ ]:
student_schema = Object(
    # This what will appear in your output. It's what the fields below will be nested under.
    # It should be the parent of the fields below.
    id="student",
    
    # Natural language description about your object
    description="Information about a student",
    
    # Fields you'd like to capture from a piece of text about your object.
    attributes=[
        Text(
            id="name",
            description="The name of a student.",
        ),
        Number(
            id="score",
            description="The score of the student.",
        )
    ],
    
    # Examples help go a long way with telling the LLM what you need
    examples=[
        ("In the Math exam, Mike scored 97 and Lucy got 88.", [{"name": "Mike", "score": 97}, {"name": "Lucy", "score": 88}])
    ]
)

In [ ]:
extraction_chain = create_extraction_chain(llm, student_schema)

In [ ]:

text = """
    Ms. Walsh just submitted the students report. 
    In the Physics final exam, Alex got 77, and Tom got 100. 
    Congratulations to Tom.
"""
output = extraction_chain.predict_and_parse(text=(text))["data"]

json_dump(output)